<a href="https://colab.research.google.com/github/jesspeck/ml_opensource_stuff/blob/main/FineTuning_Your_Own_GPT_for_Fun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get the Data (if you need to)

In [ ]:
!pip install pdfminer.six&> /dev/null

In [ ]:
# convert pdf to html

# import libraries
import pdfminer
from pdfminer.high_level import extract_text

# convert pdf to html
html = extract_text('/content/searchqualityevaluatorguidelines.pdf')

# print the html
print(html)

In [ ]:
# install tesseract
!sudo apt install tesseract-ocr&> /dev/null
!sudo apt install libtesseract-dev&> /dev/null

# install pytesseract
!pip install pytesseract&> /dev/null

In [ ]:
import pandas as pd
import pytesseract
from PIL import Image
import requests
from io import BytesIO


- [Webpage + Quality Justification Sheet](https://docs.google.com/spreadsheets/d/e/2PACX-1vQPIVCeDB9JsDOqhI7WsxqWslWDACFFDaxIyMtFV6dqivRNNcVKCPOTNpS1vj2rDMrddUHyu_1qz8or/pubhtml?gid=1622036834&single=true)
- [Query matching sheet](https://docs.google.com/spreadsheets/d/e/2PACX-1vS0DoEEMfv2PN3I5b8Txk1R9jiIz6ym8qrBjCxknlfISkOq83t8DbwEpKTQdp1LkgrE8B_b3SyTMCRp/pubhtml)

In [ ]:

df = pd.read_csv('data2.csv')

# get the links
links = df['Link']

# for each link, get the image and describe it
for link in links:
    # get the image link
    # <a class=""c28"" href=""https://www.google.com/url?q=https://guidelines.raterhub.com/images/TpuszNx1UfllzMVPr107.jpg&amp;sa=D&amp;source=editors&amp;ust=1679073266641779&amp;usg=AOvVaw1kmTlPECrg_0S2lfFmko9H
    try:
        img_link = link.split('q=')[1].split('&')[0]
    except:
        img_link = link
    # get the image file name
    #img_file = img_link.split('/')[-1]
    try:
        response = requests.get(img_link)
        img = Image.open(BytesIO(response.content))
    except:
        img = img_link
    # open the image file
    # img = Image.open(img_file)
    # describe the image
    try:
        description = pytesseract.image_to_string(img)
    except:
        description = img
    # add the description to the link column
    df.loc[df['Link'] == link, 'Link'] = description
    print(description)


In [ ]:
# convert df to json!

In [ ]:
df2 = pd.DataFrame()

In [ ]:
# content as prompt

df2['prompt'] = 'rate the following content according to quality rater guidelines' + df['Link']
df2['completion'] = df['Webpage'] + ' ' + df['Quality Justification'] + ' ' + df['PQ rating'] 
# now we are going to make some json "prompt": "", "response": ""} files from our content


In [ ]:
df2.to_json('data_content_as_prompt.json', orient='records', lines=True)

## Import your JSON

In [ ]:
# replace with whatever you wish

json = '/content/data_content_as_prompt.json'

## Open AI Time

In [ ]:
import openai


In [ ]:
!set OPENAI_API_KEY='your_key_here'

In [ ]:
!openai tools fine_tunes.prepare_data -f /content/data_content_as_prompt.json

In [ ]:
jsonfile = r'/content/data_content_as_prompt_prepared.jsonl'

In [ ]:
openai.api_key = 'your_key_here'

In [ ]:

upload_response = openai.File.create(
  file=open(jsonfile, "rb"),
  purpose='fine-tune'
)


In [ ]:
file_id = upload_response.id


In [ ]:
upload_response


In [ ]:
davinci_response = openai.FineTune.create(training_file=file_id, model="davinci",suffix="rater_guideline_model")

#%%

In [ ]:
davinci_response

In [ ]:
davinci_response.events[-1]


In [ ]:
# model_name_here should be replaced by something that looks like "davinci:ft-personal-2023-03-01-23-34-47"

fine_tuned_model = 'model_name_here'


In [ ]:
!pip install trafilatura&> /dev/null

In [ ]:
from trafilatura import fetch_url, extract

url = 'https://www.localseoguide.com/organic-keyword-research-how-to-use-n-grams-to-find-new-opportunities/'

downloaded = fetch_url(url)

result = extract(downloaded)



result = result[:1400]
print(result)

In [ ]:
context = 'Ended Content. Now, According to quality rater guidelines, describe the above content, give it a quality rating, and justify that rating: '

In [ ]:
prompt = result + context

In [ ]:
answer = openai.Completion.create(
  model=fine_tuned_model,
  prompt=prompt,
  max_tokens=500,
  temperature=0.7
)
answer['choices'][0]['text']

In [ ]:
answer = openai.Completion.create(
  model='davinci',
  prompt=prompt,
  max_tokens=500,
  temperature=0.7
)
answer['choices'][0]['text']